In [1]:
from sqlalchemy import (
    cast, Column, create_engine, func, INTEGER, NUMERIC, REAL, Table, TEXT,
    MetaData)
from sqlalchemy.engine.base import Connection
from sqlalchemy.sql import and_
from sqlalchemy.sql.expression import Select, ColumnClause, Join

In [2]:
from contextlib import suppress

In [3]:
meta = MetaData()

## Creating tables

In [4]:
broadcast_events = Table(
    'broadcast_events', meta,
    Column('time', TEXT),
    Column('change', TEXT),
    Column('point', TEXT),
    Column('namespace', TEXT),
    Column('key', TEXT),
    Column('value', TEXT)
)

broadcast_states = Table(
    'broadcast_states', meta,
    Column('point', TEXT, primary_key=True),
    Column('namespace', TEXT, primary_key=True),
    Column('key', TEXT, primary_key=True),
    Column('value', TEXT)
)

broadcast_states_checkpoints = Table(
    'broadcast_states_checkpoints', meta,
    Column('id', INTEGER, primary_key=True),
    Column('point', TEXT, primary_key=True),
    Column('namespace', TEXT, primary_key=True),
    Column('key', TEXT, primary_key=True),
    Column('value', TEXT)
)

inheritance = Table(
    'inheritance', meta,
    Column('namespace', TEXT, primary_key=True),
    Column('inheritance', TEXT)
)

suite_params = Table(
    'suite_params', meta,
    Column('key', TEXT, primary_key=True),
    Column('value', TEXT)
)

suite_params_checkpoints = Table(
    'suite_params_checkpoints', meta,
    Column('id', INTEGER, primary_key=True),
    Column('key', TEXT, primary_key=True),
    Column('value', TEXT)
)

suite_template_vars = Table(
    'suite_template_vars', meta,
    Column('key', TEXT, primary_key=True),
    Column('value', TEXT)
)

task_jobs = Table(
    'task_jobs', meta,
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('submit_num', INTEGER, primary_key=True),
    Column('is_manual_submit', INTEGER),
    Column('try_num', INTEGER),
    Column('time_submit', TEXT),
    Column('time_submit_exit', TEXT),
    Column('submit_status', INTEGER),
    Column('time_run', TEXT),
    Column('time_run_exit', TEXT),
    Column('run_signal', TEXT),
    Column('run_status', INTEGER),
    Column('user_at_host', TEXT),
    Column('batch_sys_name', TEXT),
    Column('batch_sys_job_id', TEXT)
)

task_events = Table(
    'task_events', meta,
    Column('name', TEXT),
    Column('cycle', TEXT),
    Column('time', TEXT),
    Column('submit_num', INTEGER),
    Column('event', TEXT),
    Column('message', TEXT)
)

task_action_timers = Table(
    'task_action_timers', meta,
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('ctx_key', TEXT, primary_key=True),
    Column('ctx', TEXT),
    Column('delays', TEXT),
    Column('num', INTEGER),
    Column('delay', TEXT),
    Column('timeout', TEXT)
)

checkpoint_id = Table(
    'checkpoint_id', meta,
    Column('id', INTEGER, primary_key=True),
    Column('time', TEXT),
    Column('event', TEXT)
)

task_late_flags = Table(
    'task_late_flags', meta,
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('value', INTEGER)
)

task_outputs = Table(
    'task_outputs', meta,
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('outputs', INTEGER)
)

task_pool = Table(
    'task_pool', meta,
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('spawned', INTEGER),
    Column('status', TEXT),
    Column('is_held', INTEGER)
)

task_pool_checkpoints = Table(
    'task_pool_checkpoints', meta,
    Column('id', INTEGER, primary_key=True),
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('spawned', INTEGER),
    Column('status', TEXT),
    Column('is_held', INTEGER)
)

task_states = Table(
    'task_states', meta,
    Column('name', TEXT, primary_key=True),
    Column('cycle', TEXT, primary_key=True),
    Column('time_created', TEXT),
    Column('time_updated', TEXT),
    Column('submit_num', INTEGER),
    Column('status', TEXT)
)

task_timeout_timers = Table(
    'task_timeout_timers', meta,
    Column('cycle', TEXT, primary_key=True),
    Column('name', TEXT, primary_key=True),
    Column('timeout', REAL)
)

xtriggers = Table(
    'xtriggers', meta,
    Column('signature', TEXT, primary_key=True),
    Column('results', REAL)
)

## Some meta operations...

In [20]:
xtriggers_table = meta.tables['xtriggers']

In [21]:
print([x.name for x in xtriggers_table.columns])

['signature', 'results']


In [22]:
signature_column = xtriggers_table.columns['signature']

Now create a new `xtriggers_fake` table with just one column copied from the original table...

In [44]:
meta.remove(xtriggers)
c = Column(signature_column.name, signature_column.type, primary_key=signature_column.primary_key)
fake_xtriggers = Table(xtriggers.name, meta, c)
fake_xtriggers
# fake_xtriggers.create()

Table('xtriggers', MetaData(bind=None), Column('signature', TEXT(), table=<xtriggers>, primary_key=True, nullable=False), schema=None)

In [48]:
fake_xtriggers
fake_xtriggers.columns['signature'].remove()
fake_xtriggers

AttributeError: Neither 'Column' object nor 'Comparator' object has an attribute 'remove'